# Reconhecimento de Movimentos Utilizando uma Rede Feedforward (FNN)
## *Fine Tuning*

## Preparação dos *Datasets*

In [1]:
import pandas as pd
from keras.utils import *

# Carregar os datasets
X_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", header=None, sep="\s+")
y_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", header=None)

X_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", sep="\s+", header=None)
y_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7352, 561), (7352, 7), (2947, 561), (2947, 7))

## Criar o Modelo

In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *

def build_model(hp):
    # Criação do objeto do Modelo
    modelo = Sequential()
    modelo.add(Dense(hp.Int("neurons_dense_1", min_value=1, max_value=600), activation=hp.Choice("activation_dense_1", ["relu", "sigmoid"])))
    modelo.add(Dense(hp.Int("neurons_dense_2", min_value=1, max_value=600), activation=hp.Choice("activation_dense_2", ["relu", "sigmoid"])))
    modelo.add(Dense(7, activation="softmax"))
    modelo.compile(loss=hp.Choice("loss", ["binary_crossentropy", "sparse_categorical_crossentropy"]), optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.0001, 0.001, 0.01])), metrics=["accuracy"])
    return modelo

## Fine Tuning

In [4]:
from keras_tuner import *

tuner = BayesianOptimization(hypermodel=build_model, objective='accuracy')
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=10)

Reloading Tuner from .\untitled_project\tuner0.json


In [5]:
best_model = tuner.get_best_models()[0]
best_model.summary()

C:\Users\daniel.a.goncalves\Desktop\tedi\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 113)                 │          63,506 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 188)                 │          21,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │           1,323 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 86,261 (336.96 KB)

 Trainable params: 86,261 (336.96 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print(f"Neurónios dense 1: {best_hyperparameters.get('neurons_dense_1')}")
print(f"Função de Ativação dense 1: {best_hyperparameters.get('activation_dense_1')}")
print(f"Neurónios dense 2: {best_hyperparameters.get('neurons_dense_2')}")
print(f"Função de Ativação dense 2: {best_hyperparameters.get('activation_dense_2')}")
print(f"Função de Perda: {best_hyperparameters.get('loss')}")
print(f"Learning Rate: {best_hyperparameters.get('learning_rate')}")

Neurónios dense 1: 113
Função de Ativação dense 1: sigmoid
Neurónios dense 2: 188
Função de Ativação dense 2: sigmoid
Função de Perda: sparse_categorical_crossentropy
Learning Rate: 0.001
